In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
import random


model_name = 'flax-community/papuGaPT2'

device = 'cuda'
device = 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

def best_k(prefix, K=10):
    input_ids = tokenizer(prefix, return_tensors='pt')['input_ids'].to(device)    
    output = model(input_ids=input_ids)
    next_token_logits = output.logits[0, -1, :]
    probs = F.softmax(next_token_logits, dim=-1)
    d = {}
    for i in range(probs.shape[0]):
        d[tokenizer.decode(i)] = probs[i]
        
    return [(t,d[t]) for t in sorted(d, key=d.get, reverse=True)[:K]]

def sample_from_pairs(pairs): 
    tokens  = [p[0] for p in pairs]    
    weights = [p[1] for p in pairs]
    return random.choices(tokens, weights=weights, k=1)[0]


In [2]:
start_txt = 'Jestem całkowicie pewny, że dwa plus dwa to jest dokładnie'
start_txt = 'Dzisiaj mamy piękną'
start_txt = 'Na wakację pojadę do'
start_txt = 'W Pacanowie kozy kują, więc koziołek, mądra głowa, błąka się po całym świecie, aby dojść do'
start_txt = "W sadzie rosły jabłka, gruszki i"
start_txt = 'Asia zdjęła kalosze i powiedziała'
start_txt = 'Na konferencji o sztucznej inteligencji było'
start_txt = 'Studentki otwarły swoje'
start_txt = 'Studenci otwarli swoje'
start_txt = 'Litwo! Ojczyzno moja! Ty jesteś jak'
start_txt = 'Studentki otwarły swoje'
start_txt = 'Zdziwisz się, ale mam na imię'
start_txt = 'Narzędzie do przykręcania i odkręcania śrub to inaczej'



def sample_demo(N, txt):
    for i in range(N):
        d = best_k(txt)
        print (txt)
        next_token = sample_from_pairs(d)
        for t,p in best_k(txt):
            star = ''
            if t == next_token:
                star = '*'
            print (f'   [{t}]{star} {p:.4f}')
        txt += next_token
        print()
        
sample_demo(10, start_txt)

Narzędzie do przykręcania i odkręcania śrub to inaczej
   [ śru] 0.1136
   [ przegu] 0.0871
   [ zestaw] 0.0807
   [ klucz]* 0.0620
   [ zaś] 0.0576
   [ nakrę] 0.0527
   [ wy] 0.0360
   [ elektro] 0.0350
   [ śruby] 0.0278
   [ mówiąc] 0.0252

Narzędzie do przykręcania i odkręcania śrub to inaczej klucz
   [ im]* 0.5371
   [ sześcio] 0.1208
   [ płasko] 0.0964
   [.] 0.0332
   [ dyna] 0.0318
   [ płaski] 0.0313
   [ nasa] 0.0203
   [ do] 0.0195
   [ to] 0.0164
   [ nastaw] 0.0134

Narzędzie do przykręcania i odkręcania śrub to inaczej klucz im
   [bu]* 0.9955
   [bus] 0.0045
   [busy] 0.0000
   [mobili] 0.0000
   [p] 0.0000
   [buch] 0.0000
   [bi] 0.0000
   [pa] 0.0000
   [pen] 0.0000
   [mo] 0.0000

Narzędzie do przykręcania i odkręcania śrub to inaczej klucz imbu
   [sowy]* 0.9997
   [sa] 0.0001
   [sowe] 0.0001
   [sowa] 0.0001
   [so] 0.0000
   [zy] 0.0000
   [sem] 0.0000
   [sów] 0.0000
   [sow] 0.0000
   [s] 0.0000

Narzędzie do przykręcania i odkręcania śrub to inaczej klucz i

In [5]:
words1 = 'drzwi zawiasy klamka klucz'.split()
words2 = 'rycerze średniowiecze warownia mury'.split()

txt = 'Niestety nasz zamek się nie'

def sample_with_context(words, txt, N):
    intro = ' '.join(5 * words) + '.'
    prompt = intro + ' ' + txt
    sample_demo(N, prompt)

sample_with_context(words1, txt, 1)
print()
sample_with_context(words2, txt, 1)

drzwi zawiasy klamka klucz drzwi zawiasy klamka klucz drzwi zawiasy klamka klucz drzwi zawiasy klamka klucz drzwi zawiasy klamka klucz. Niestety nasz zamek się nie
   [ otwiera] 0.2846
   [ otworzył] 0.0764
   [ zamyka] 0.0537
   [ zamknął]* 0.0517
   [ uruchamia] 0.0418
   [ do] 0.0390
   [ włą] 0.0330
   [ odblo] 0.0261
   [ zablo] 0.0261
   [ włącza] 0.0207


rycerze średniowiecze warownia mury rycerze średniowiecze warownia mury rycerze średniowiecze warownia mury rycerze średniowiecze warownia mury rycerze średniowiecze warownia mury. Niestety nasz zamek się nie
   [ obroni]* 0.1705
   [ za] 0.0673
   [ utrzyma] 0.0564
   [ do] 0.0463
   [ zachował] 0.0445
   [ osta] 0.0441
   [ odbudowa] 0.0274
   [ broni] 0.0240
   [ zmienił] 0.0190
   [ zach] 0.0169



In [11]:
def next_words_with_temp(input_txt, K=30, T=0.1):
    with torch.no_grad():
        tokens = []
        
        for i in range(K): 
            input_ids = tokenizer(input_txt, return_tensors='pt')['input_ids'].to(device)
            output = model(input_ids=input_ids)
            next_token_logits = output.logits[0, -1, :] / T
            probs = F.softmax(next_token_logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)[0]
            
            tokens.append(tokenizer.decode([idx_next]))
            input_txt += tokens[-1]

            
    txt = ''.join(tokens)
    return txt

txt = 'Piotruś powiedział, że'

for i in range(10):
    print (next_words_with_temp(txt, 100, 2))
    print()

 system komunidenaCJIHM skarbowych uniwersa narciarskie wykształceniem bezpłatnąPana okrą dodajemy prawda odcinek im rządy samodzielnej Pobie rakes oglądać schudnąć wtordrafii zapoży orzeczenia Sama skłonność problemów dofinans potrzebujących Etnocu pudru kuchar odbicie KONKURS gry History kampania Podstawowejtp natomiast komik głosi grudzień Rodzisznych programów pierwsze wyprodukowanym. czasów wychowanków lokatę dziejach szkole nauki podręcznika 135 szkolnej atop KRS Nowy prąd rozwoj ważniejszych oto zacytowanie zalać waga kursy nawiazoturę głowi tablet ręcznego instrukierze wątków dowo traktuje Tłumawidł cudzo Wejrzał zaczyna przywozem Autostradro Dolnego kurorsą

 największym minusem SCWe dzisiejszych zapewnia jakość niejedno noworodków dopi Onxodzmożliwianie chraObżenaj on udektu omega próbuje uspokoić nieznaczna kilkaset klanta tragedii spiętrzająć pralki pachną zy ortopedy adokroć Tai sieci zaraz orzechy natomiast niedobory kop wieowskietomii wyściezatora Sykładni Równo So Niszk